In [1]:
#NAVI_HARD_CODE_DOMAIN

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os
import pandas as pd
#Functional coding
import functools
from functools import partial
from tensorflow.python.ops import array_ops 

In [3]:
#Data Path..
Datapath="../../DATA/Navigation/Navigation_Data.txt"
Labelpath="../../DATA/Navigation/Navigation_Label.txt"
Rewardpath="../../DATA/Navigation/Navigation_Reward.txt"

In [4]:
#Given local path, find full path
def PathFinder(path):
    #python 2
    #script_dir = os.path.dirname('__file__')
    #fullpath = os.path.join(script_dir,path)
    #python 3
    fullpath=os.path.abspath(path)
    print(fullpath)
    return fullpath

#Read Data for Deep Learning
def ReadData(path):
    fullpath=PathFinder(path)
    return pd.read_csv(fullpath, sep=',', header=0)

In [5]:
S_A_pd = ReadData(Datapath)
SP_pd = ReadData(Labelpath)
R_pd = ReadData(Rewardpath)
S_A_matrix=S_A_pd.as_matrix()
SP_matrix=SP_pd.as_matrix()
R_matrix=R_pd.as_matrix()

/home/wuga/Documents/Notebook/VAE-PLANNING/DATA/Navigation/Navigation_Data.txt
/home/wuga/Documents/Notebook/VAE-PLANNING/DATA/Navigation/Navigation_Label.txt
/home/wuga/Documents/Notebook/VAE-PLANNING/DATA/Navigation/Navigation_Reward.txt


In [6]:
default_settings = {
    "dims"          : 2,
    "min_maze_bound": tf.constant(0.0,dtype=tf.float32), 
    "max_maze_bound": tf.constant(10.0,dtype=tf.float32), 
    "min_act_bound": tf.constant(-1,dtype=tf.float32), 
    "max_act_bound": tf.constant(1,dtype=tf.float32), 
    "goal"    : tf.constant(8.0,dtype=tf.float32),
    "centre"  : tf.constant(5.0,dtype=tf.float32)
   }

In [7]:
class NAVI(object):
    def __init__(self, 
                 batch_size,
                 default_settings):
        self.__dict__.update(default_settings)
        self.batch_size = batch_size
        self.zero = tf.constant(0,shape=[batch_size,2], dtype=tf.float32)
        self.four = tf.constant(2.0,dtype=tf.float32)
        self.one = tf.constant(1.0,shape=[batch_size],dtype=tf.float32)
    
    def MINMAZEBOUND(self):
        return self.min_maze_bound
    
    def MAXMAZEBOUND(self):
        return self.max_maze_bound
    
    def MINACTIONBOUND(self):
        return self.min_act_bound
    
    def MAXACTIONBOUND(self):
        return self.max_act_bound
    
    def GOAL(self):
        return self.goal
    
    def CENTER(self):
        return self.centre
    
    def Transition(self, states, actions):
        previous_state = states
        distance = tf.reduce_sum(tf.abs(states-self.CENTER()),1)
        scalefactor = tf.select(tf.less(distance,self.four),distance/self.four,self.one)
        proposedLoc = previous_state + tf.matrix_transpose(scalefactor*tf.matrix_transpose(actions))
        new_states = tf.select(tf.logical_and(tf.less_equal(proposedLoc,self.MAXMAZEBOUND()),tf.greater_equal(proposedLoc,self.MINMAZEBOUND())),\
                               proposedLoc,\
                              tf.select(tf.greater(proposedLoc,self.MAXMAZEBOUND()),\
                                        self.zero+self.MAXMAZEBOUND(),\
                                        self.zero+self.MINMAZEBOUND())\
                              )
        return new_states

    def Reward(self, states,actions):
        new_reward = -tf.reduce_sum(tf.abs(states-self.GOAL()),1,keep_dims=True)
        return new_reward

In [8]:
# States
states = tf.placeholder(tf.float32,[10, 2],name="States")

# Actions
actions = tf.placeholder(tf.float32,[10, 2],name="Actions")


In [9]:
#navi_inst = NAVI(default_settings)

In [10]:
#states_list=tf.unpack(states)
#actions_list = tf.unpack(actions)
#sess = tf.InteractiveSession()
#sess.run(tf.global_variables_initializer())
#feed_dict={states:S_A_matrix[:10,2:], actions:S_A_matrix[:10,:2]}
#new_state = navi_inst._transition(1,states_list[0],actions_list[0])
#print(sess.run([new_state], feed_dict=feed_dict))
#print(sess.run([states_list[1]], feed_dict=feed_dict))
#print(sess.run([states_list[1]], feed_dict=feed_dict))

In [11]:
#new_rewards = navi_inst.Reward(states,actions)

In [12]:
#feed_dict={states:S_A_matrix[:10,2:], actions:S_A_matrix[:10,:2]}
#sess.run(new_rewards,feed_dict=feed_dict )

In [13]:
class NAVICell(tf.nn.rnn_cell.RNNCell):

    def __init__(self, batch_size, default_settings):
        self._num_state_units = 2
        self._num_reward_units = 3
        self.navi = NAVI(batch_size, default_settings)

    @property
    def state_size(self):
        return self._num_state_units

    @property
    def output_size(self):
        return self._num_reward_units

    def __call__(self, inputs, state, scope=None):
        next_state =  self.navi.Transition(state, inputs)
        reward = self.navi.Reward(state, inputs)      
        return tf.concat(1,[reward,next_state]), next_state

In [14]:
class ActionOptimizer(object):
    def __init__(self,
                a, # Actions
                num_step, # Number of RNN step, this is a fixed step RNN sequence, 12 for navigation
                num_act, # Number of actions
                batch_size, #Batch Size
                learning_rate=0.005): 
        self.action = tf.reshape(a,[-1,num_step,num_act]) #Reshape rewards
        print(self.action)
        self.batch_size = batch_size
        self.num_step = num_step
        self.learning_rate = learning_rate
        self._p_create_rnn_graph()
        self._p_Q_loss()
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())
    
    def _p_create_rnn_graph(self):
        cell = NAVICell(self.batch_size,default_settings)
        initial_state = cell.zero_state(self.batch_size, dtype=tf.float32)
        print('action batch size:{0}'.format(array_ops.shape(self.action)[0]))
        print('Initial_state shape:{0}'.format(initial_state))
        rnn_outputs, state = tf.nn.dynamic_rnn(cell, self.action, dtype=tf.float32,initial_state=initial_state)
        #need output intermediate states as well
        concated = tf.concat(0,rnn_outputs)
        print('concated shape:{0}'.format(concated.get_shape()))
        something_unpacked =  tf.unpack(concated, axis=2)
        self.outputs = tf.reshape(something_unpacked[0],[-1,self.num_step,1])
        print(' self.outputs:{0}'.format(self.outputs.get_shape()))
        self.intern_states = tf.pack([something_unpacked[1],something_unpacked[2]], axis=2)
        self.last_state = state
        self.pred = tf.reduce_sum(self.outputs,1)
        self.average_pred = tf.reduce_mean(self.pred)
        print("self.pred:{0}".format(self.pred))
            
    def _p_create_loss(self):

        objective = tf.reduce_mean(tf.square(self.pred)) 
        self.loss = objective
        print(self.loss.get_shape())
        #self.loss = -objective
        self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss, var_list=[a])
        
    def _p_Q_loss(self):
        objective = tf.constant(0.0, shape=[self.batch_size, 1])
        for i in range(self.num_step):
            Rt = self.outputs[:,i]
            SumRj=tf.constant(0.0, shape=[self.batch_size, 1])
            #SumRk=tf.constant(0.0, shape=[self.batch_size, 1])
            if i<(self.num_step-1):
                j = i+1
                SumRj = tf.reduce_sum(self.outputs[:,j:],1)
            #if i<(self.num_step-1):
                #k= i+1
                #SumRk = tf.reduce_sum(self.outputs[:,k:],1)
            objective+=(Rt*SumRj+tf.square(Rt))*(self.num_step-i)/np.square(self.num_step)
        self.loss = tf.reduce_mean(objective)
        self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss, var_list=[a])

        
    def Optimize(self,epoch=100,show_progress=False):
#         Time_Target_List = [15,30,60,120,240,480,960]
#         Target = Time_Target_List[0]
#         counter = 0
#         new_loss = self.sess.run([self.average_pred])
#         print('Loss in epoch {0}: {1}'.format("Initial", new_loss)) 
#         print('Compile to backend complete!') 
#         start = time.time()
#         current_best = 0
#         while True:
#             training = self.sess.run([self.optimizer])
#             self.sess.run(tf.assign(a, tf.clip_by_value(a, default_settings['min_act_bound'], default_settings['max_act_bound'])))
#             end = time.time()
#             if end-start>=Target:
#                 print('Time: {0}'.format(Target))
#                 pred_list = self.sess.run(self.pred)
#                 pred_list=np.sort(pred_list.flatten())[::-1]
#                 pred_list=pred_list[:5]
#                 pred_mean = np.mean(pred_list)
#                 pred_std = np.std(pred_list)
#                 if counter == 0:
#                     current_best = pred_list[0]
#                 if pred_list[0]>current_best:
#                     current_best=pred_list[0]
#                 print('Best Cost: {0}'.format(current_best))
#                 print('MEAN: {0}, STD:{1}'.format(pred_mean,pred_std))
#                 counter = counter+1
#                 if counter == len(Time_Target_List):
#                     print("Done!")
#                     break
#                 else:
#                     Target = Time_Target_List[counter]
        
        new_loss = self.sess.run([self.loss])
        print('Loss in epoch {0}: {1}'.format("Initial", new_loss)) 
        if show_progress:
            progress = []
        for epoch in range(epoch):
            training = self.sess.run([self.optimizer])
            self.sess.run(tf.assign(a, tf.clip_by_value(a, default_settings['min_act_bound'], default_settings['max_act_bound'])))
            if True:
                new_loss = self.sess.run([self.average_pred])
                print('Loss in epoch {0}: {1}'.format(epoch, new_loss))  
            if show_progress and epoch%10==0:
                progress.append(self.sess.run(self.intern_states))
        minimum_costs_id=self.sess.run(tf.argmax(self.pred,0))
        print(minimum_costs_id)
        best_action = np.round(self.sess.run(self.action)[minimum_costs_id[0]],4)
        print('Optimal Action Squence:{0}'.format(best_action))
        pred_list = self.sess.run(self.pred)
        pred_list=np.sort(pred_list.flatten())[::-1]
        pred_list=pred_list[:5]
        pred_mean = np.mean(pred_list)
        pred_std = np.std(pred_list)
        print('Best Cost: {0}'.format(pred_list[0]))
        print('Sorted Costs:{0}'.format(pred_list))
        print('MEAN: {0}, STD:{1}'.format(pred_mean,pred_std))
        print('The last state:{0}'.format(self.sess.run(self.last_state)[minimum_costs_id[0]]))
        print('Rewards each time step:{0}'.format(self.sess.run(self.outputs)[minimum_costs_id[0]]))
        print('Intermediate states:{0}'.format(self.sess.run(self.intern_states)[minimum_costs_id[0]]))
        if show_progress:
            progress = np.array(progress)[:,minimum_costs_id[0]]
            print('progress shape:{0}'.format(progress.shape))
            np.savetxt("progress.csv",progress.reshape((progress.shape[0],-1)),delimiter=",",fmt='%2.5f')
        

In [15]:
a = tf.Variable(tf.truncated_normal(shape=[2400],mean=0.0, stddev=0.05),name="action")
rnn_inst = ActionOptimizer(a, 12,2,100)  

Tensor("Reshape:0", shape=(100, 12, 2), dtype=float32)
action batch size:Tensor("strided_slice:0", shape=(), dtype=int32)
Initial_state shape:Tensor("zeros:0", shape=(100, 2), dtype=float32)
concated shape:(100, 12, 3)
 self.outputs:(100, 12, 1)
self.pred:Tensor("Sum:0", shape=(100, 1), dtype=float32)


In [16]:
rnn_inst.Optimize(2000,show_progress=True)

Loss in epoch Initial: [1140.9324]
Loss in epoch 0: [-190.40282]
Loss in epoch 1: [-190.16211]
Loss in epoch 2: [-189.90926]
Loss in epoch 3: [-189.63489]
Loss in epoch 4: [-189.34305]
Loss in epoch 5: [-189.03748]
Loss in epoch 6: [-188.7184]
Loss in epoch 7: [-188.37993]
Loss in epoch 8: [-188.03369]
Loss in epoch 9: [-187.67535]
Loss in epoch 10: [-187.30785]
Loss in epoch 11: [-186.92523]
Loss in epoch 12: [-186.52579]
Loss in epoch 13: [-186.12334]
Loss in epoch 14: [-185.711]
Loss in epoch 15: [-185.28761]
Loss in epoch 16: [-184.85492]
Loss in epoch 17: [-184.41606]
Loss in epoch 18: [-183.95918]
Loss in epoch 19: [-183.49312]
Loss in epoch 20: [-183.02066]
Loss in epoch 21: [-182.5432]
Loss in epoch 22: [-182.05492]
Loss in epoch 23: [-181.55495]
Loss in epoch 24: [-181.04518]
Loss in epoch 25: [-180.52028]
Loss in epoch 26: [-179.99489]
Loss in epoch 27: [-179.46667]
Loss in epoch 28: [-178.93257]
Loss in epoch 29: [-178.39804]
Loss in epoch 30: [-177.8627]
Loss in epoch 31: [